In [2]:
from pathlib import Path
import pickle
import numpy as np
from scipy import signal
import cv2

In [3]:
video_path = Path('data/jaad/JAAD_clips/video_0021.mp4')
keypoints = Path('data/processed/jaad_database.pkl')
keypoints.exists()

True

In [4]:
with open(str(keypoints), 'rb') as f:
    db = pickle.load(f)

In [4]:
skeletons = db['video_0021']['ped_annotations']['0_21_92b']['skeleton_keypoints']

NameError: name 'db' is not defined

In [1]:
skeletons

NameError: name 'skeletons' is not defined

In [7]:
stack = np.vstack(skeletons[48:])[:, :2].reshape(-1, 50).astype(np.int32)
stack_smooth = stack.copy()



In [8]:
stack[1] - stack_smooth[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   61,
         39,    0,    1,   61,   39,    0,    1,   61,   40,    0,    1,
         61,   40,    0,    1,    1,    1,    1,    0,  368,  248,    1,
          1,   62,   38, -368, -221,  368,  249, -122,  -83,  368,  249,
          1,    1,    0,    0,    0,    1], dtype=int32)

In [9]:
with open('/Users/seba/Downloads/stack.npy', 'wb') as f:
    np.save(f, stack)
with open('/Users/seba/Downloads/stack_smooth.npy', 'wb') as f:
    np.save(f, stack_smooth)

In [10]:
stack

array([[   0,    0,    0, ...,    0, 1075,  721],
       [   0,    0,    0, ...,    0, 1074,  724],
       [   0,    0,    0, ...,    0, 1074,  726],
       ...,
       [1844,  510, 1841, ...,    0,    0,    0],
       [1902,  499, 1902, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int32)

In [33]:
window_length, polyorder = 7, 3
cleaned_points = []

for i in range(stack.shape[1]):
    stack_smooth[:, i] = signal.savgol_filter(stack[:, i], window_length, polyorder)
    
for i in range(stack_smooth.shape[0]):
    points = list(zip(stack_smooth[i,::2], stack_smooth[i,1::2]))
    cleaned_points.append(points)
    
circle_color, line_color = (0,255,255), (0,255,0)

cap = cv2.VideoCapture(str(video_path))

n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = float(cap.get(cv2.CAP_PROP_FPS))

ok, frame = cap.read()
frame_height, frame_width = frame.shape[:2]

h = 1080
w = 1920

# Define the output
out_path = '/Users/seba/Downloads/output_stack_smooth.mp4'
output = cv2.VideoWriter(out_path, 0, fps, (w, h))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = None
(f_h, f_w) = (h, w)
zeros = None

# There are 25 points in the skeleton
pairs = [[17,18], # head
         [17,5],[17,6], # sholders
         [5,7],[7,9],[6,8],[8,10], # arms
         [17,11],[17,12],[11,12], # hips
         [11,13],[13,15],[12,14],[14,16]] # legs

frame_number = 0




while True:
    ok, frame = cap.read()

    if not ok:
        break

    frame = cv2.resize(frame, (w, h), cv2.INTER_AREA)
    frame_copy = np.copy(frame)

    if frame_number >= 48 and frame_number <= 170:
        points = cleaned_points[frame_number-48]
        for i in range(len(points)):
            xy = tuple(np.array([points[i][0], points[i][1]], int))
            cv2.circle(frame_copy, xy, 2, circle_color, -1)

        for pair in pairs:
            partA = pair[0]
            partB = pair[1]
            cv2.line(frame_copy, points[partA], points[partB], line_color, 1, lineType=cv2.LINE_AA)


    if writer is None:
        writer = cv2.VideoWriter(out_path, fourcc, fps,
                                 (f_w, f_h), True)
        zeros = np.zeros((f_h, f_w), dtype="uint8")

    writer.write(frame_copy)

    cv2.imshow('Output-Skeleton', frame_copy)

    k = cv2.waitKey(100)
    if k == 27:
        break

    frame_number+=1

cv2.destroyAllWindows()
writer.release()

OpenCV: FFMPEG: tag 0x00000000/'????' is not supported with codec id 13 and format 'mp4 / MP4 (MPEG-4 Part 14)'
[mp4 @ 0x282a6c000] Could not find tag for codec rawvideo in stream #0, codec not currently supported in container


In [23]:
cv2

<module 'cv2' from '/Users/seba/miniforge3/envs/pedestrians/lib/python3.9/site-packages/cv2.cpython-39-darwin.so'>

In [14]:
frame_number-48

131

In [35]:
from sklearn.base import BaseEstimator, TransformerMixin
class InterpolateKeypoints(BaseEstimator, TransformerMixin):
    """Interpolates between the specified keypoints.
    I.e., creates intermediate points between connect_keypoints which
    can help enrichen the point clouds inputted into the Persistence
    calculator.
    Parameters
    ----------
    connect_keypoints : array of tuples
        Each tuple should have the form (from, to) which specifies which
        points in the data to add points between.
    number_of_points : int, optional, default=2
        Specifies how many points to add between every pair of points.
    """

    def __init__(self, connect_keypoints, number_of_points=2):
        self.connect_keypoints = connect_keypoints
        self.number_of_points = number_of_points

    def fit(self, X, y=None, **fit_params):
        """Returns self unchanged, as there are no parameters to fit.
        Parameters
        ----------
        X : ignored
        y : ignored
        fit_params : ignored
        Returns
        -------
        self : unchanged
        """
        return self

    def transform(self, chunks):
        """Adds interpolated points between the specified keypoints to the input.
        Parameters
        ----------
        chunks : array-like
            shape = [n_chunks, frames_per_chunk, n_keypoints, 2]
            Adds values to each subarray in the array.
        Returns
        -------
        chunks : array-like
            shape = [n_chunks, frames_per_chunk,
                n_keypoints + len(connect_keypoints) * number_of_points
                , 2]
        """
        return np.array([self._connect_keypoints(chunk) for chunk in chunks])

    def _connect_keypoints(self, chunk):
        new_number_of_keypoints = chunk.shape[1] + \
            len(self.connect_keypoints) * self.number_of_points

        connected_chunk = np.zeros((chunk.shape[0], new_number_of_keypoints, chunk.shape[2]))
        connected_chunk[:, :chunk.shape[1]] = chunk

        for i, frame in enumerate(chunk):
            for j, (from_, to) in enumerate(self.connect_keypoints):
                start_index = j * self.number_of_points + chunk.shape[1]
                intermediate_points = self._intermediate_points(
                    frame, from_, to, self.number_of_points)
                for k, points in enumerate(intermediate_points):
                    connected_chunk[i, start_index + k] = points

        return connected_chunk

    def _intermediate_points(self, frame, from_, to, number_of_points):
        from_point = frame[from_]
        to_point = frame[to]
        diff = from_point - to_point
        step = diff / (number_of_points + 1)
        step_array = np.tile(step, number_of_points).reshape(-1, step.shape[0])

        intermediate_points = to_point + step_array * \
            np.arange(1, number_of_points + 1)[:, np.newaxis]
        return intermediate_points